# Algorytmy macierzowe - zadanie nr 5 - Algorytmy permutacji macierzy rzadkich

"Proszę zbudować graf eliminacji dla macierzy rzadkiej używanej w zadaniu 4
(rzadka eliminacja) (bazując na swoim formacie macierzy rzadkiej z zadania 4)

Proszę napisać wybrany algorytm permutacji macierzy na swojej macierzy
rzadkiej:
1. Minimum degree
2. Cuthill-McKee
3. Nested-dissections

Proszę porównać czasy działania rzadkiej eliminacji Gaussa przed i po
permutacji macierzy"

Marcin Hawryluk, Norbert Wolniak <br>grupa: piątek 12:50B <hr>

In [1]:
import numpy as np
import networkx as nx
from time import time
import pandas as pd
import matplotlib.pyplot as plt
import os
from time import time
from matrix_functions import read_matrix, sparse_cholesky, convert_to_csr

## Generowanie macierzy

In [8]:
matrices  = {}
for file in os.listdir('matrices'):
    matrices[file[:2]]= read_matrix('matrices/' + file)

## Graf eliminacji

In [2]:
def elimination_graph(matrix):
    pass

"2a. Dla podmacierzy o rozmiarze <= 100 np. A[1:100][1:100] wierszy proszę
zbudować i narysować graf eliminacji G0"

"2b. Proszę uruchomić eliminację Gaussa na tym grafie G0 i narysowaćc nowe
krawędzie w G0"

## Algorytm permutacji macierzy

"3. Proszę napisać i opisać kod wybranego algorytm permutacji macierzy"

"4. Proszę uruchomić macierz permutacji dla macierzy z punktu 2"

"5a. Proszę narysować graf eliminacji G0’ dla spermutowanej małej macierz z
punktu 2"

"5b. Proszę uruchomić eliminację Gaussa na tym grafie G0’ i narysować nowe
krawędzie w G0"

"6. Proszę uruchomić swój algorytm permutacji dla całej dużej (rozmiar > 100)
macierzy z zadania 4"

## Porównanie czasów

"7. Proszę porównać czasy rzadkiej eliminacji Gaussa przed permutacją i po
permutacji dla dużej macierzy (rozmiar > 100)"

## Wnioski

* 
* 

M. Hawryluk, N. Wolniak. 2022